In [1]:
from transformers import DistilBertTokenizerFast
from datasets import *
import torch
import ast
import numpy as np

In [8]:
data = load_dataset('csv', data_files="../data/clean/sustainability-report-2020-squad-format.csv", delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/rjutr/.cache/huggingface/datasets/csv/default-6a9a3e730f68f403/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


In [9]:
data["test"] = data["test"].map(lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {"text": example["text"], "answer_start": example["answer_start"]}})
data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {"text": example["text"], "answer_start": example["answer_start"]}})
data["train"].remove_columns(["text", "answer_start"])

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'context', 'answers', 'id'],
    num_rows: 129
})

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [11]:
def tokenize_sample_data(data):
  # Tokenize
  tokenized_feature = tokenizer(
    data["question"],
    data["context"],
    max_length = 384,
    return_overflowing_tokens=True,
    stride=128,
    truncation="only_second",
    padding = "max_length",
    return_offsets_mapping=True,
  )

  # When it overflows, multiple rows will be returned for a single example.
  # The following then gets the array of corresponding the original sample index.
  sample_mapping = tokenized_feature.pop("overflow_to_sample_mapping")
  # Get the array of [start_char, end_char + 1] in each token.
  # The shape is [returned_row_size, max_length]
  offset_mapping = tokenized_feature.pop("offset_mapping")

  start_positions = []
  end_positions = []
  for i, offset in enumerate(offset_mapping):
    sample_index = sample_mapping[i]
    answers = data["answers"][sample_index]
    start_char = answers["answer_start"][0]
    end_char = start_char + len(answers["text"][0]) - 1
    # The format of sequence_ids is [None, 0, ..., 0, None, None, 1, ..., 1, None, None, ...]
    # in which question's token is 0 and contex's token is 1
    sequence_ids = tokenized_feature.sequence_ids(i)
    # find the start and end index of context
    idx = 0
    while sequence_ids[idx] != 1:
      idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1
    # Set start positions and end positions in inputs_ids
    # Note: The second element in offset is end_char + 1
    # if offset[context_start][0] > end_char or offset[context_end][1] <= start_char:
    if not (offset[context_start][0] <= start_char and end_char < offset[context_end][1]):
      # The case that answer is not inside the context
      ## Note : Some tokenizer (such as, tokenizer in rinna model) doesn't place CLS
      ## for the first token in sequence, and I then set -1 as positions.
      ## (Later I'll process rows with start_positions=-1.)
      start_positions.append(-1)
      end_positions.append(-1)
    else:
      # The case that answer is found in the context

      # Set start position
      idx = context_start
      while offset[idx][0] < start_char:
        idx += 1
      if offset[idx][0] == start_char:
        start_positions.append(idx)
      else:
        start_positions.append(idx - 1)

      # Set end position
      idx = context_end
      while offset[idx][1] > end_char + 1:
        idx -= 1
      if offset[idx][1] == end_char + 1:
        end_positions.append(idx)
      else:
        end_positions.append(idx + 1)

  # Build result
  tokenized_feature["start_positions"] = start_positions
  tokenized_feature["end_positions"] = end_positions   
  return tokenized_feature

# Run conversion
tokenized_ds = data.map(
  tokenize_sample_data,
  remove_columns=["id", "context", "question", "answers"],
  batched=True,
  batch_size=128)

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [12]:
tokenized_ds = tokenized_ds.filter(lambda x: x["start_positions"] != -1)

Filter:   0%|          | 0/129 [00:00<?, ? examples/s]

Filter:   0%|          | 0/56 [00:00<?, ? examples/s]

In [13]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'answer_start', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 129
    })
    test: Dataset({
        features: ['text', 'answer_start', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 56
    })
})

In [14]:
from transformers import AutoModelForQuestionAnswering

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = (AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased").to(device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [15]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir = "distilbert-nlb-qa",
  log_level = "error",
  num_train_epochs = 200,
  learning_rate = 7e-5,
  lr_scheduler_type = "linear",
  warmup_steps = 100,
  per_device_train_batch_size = 16,
  per_device_eval_batch_size = 16,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 150,
  save_steps = 500,
  logging_steps = 50,
  push_to_hub = False
)

In [17]:
from transformers import Trainer

trainer = Trainer(
  model = model,
  args = training_args,
  data_collator = data_collator,
  train_dataset = tokenized_ds["train"],
  eval_dataset = tokenized_ds["test"].select(range(data["test"].shape[0] // 2)),
  tokenizer = tokenizer,
)

trainer.train()

c:\Users\rjutr\miniconda3\envs\project_ds\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 1.7114, 'learning_rate': 3.5e-05, 'epoch': 33.78}
{'loss': 0.0824, 'learning_rate': 7e-05, 'epoch': 67.0}
{'loss': 0.0012, 'learning_rate': 3.5e-05, 'epoch': 100.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 2.4316065311431885, 'eval_runtime': 0.1401, 'eval_samples_per_second': 199.818, 'eval_steps_per_second': 14.273, 'epoch': 100.0}
{'loss': 0.0006, 'learning_rate': 0.0, 'epoch': 133.33}
{'train_runtime': 247.3689, 'train_samples_per_second': 104.298, 'train_steps_per_second': 0.809, 'train_loss': 0.4489036752842367, 'epoch': 133.33}


TrainOutput(global_step=200, training_loss=0.4489036752842367, metrics={'train_runtime': 247.3689, 'train_samples_per_second': 104.298, 'train_steps_per_second': 0.809, 'train_loss': 0.4489036752842367, 'epoch': 133.33})

In [18]:
trainer.evaluate()

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 2.130403518676758,
 'eval_runtime': 0.14,
 'eval_samples_per_second': 200.062,
 'eval_steps_per_second': 14.29,
 'epoch': 133.33}

In [40]:
import torch

def inference_answer(model, question, context):
  question = question
  context = context
  test_feature = tokenizer(
    question,
    context,
    max_length=318,
  )
  with torch.no_grad():
    outputs = model(torch.tensor([test_feature["input_ids"]]).to(device))
  start_logits = outputs.start_logits.cpu().numpy()
  end_logits = outputs.end_logits.cpu().numpy()
  answer_ids = test_feature["input_ids"][np.argmax(start_logits):np.argmax(end_logits)+1]
  return " ".join(tokenizer.batch_decode(answer_ids))


In [41]:
answer_pred = [inference_answer(model, data["test"]["question"][idx], data["test"]["context"][idx]) for idx in range(data["test"].shape[0])]
answer_true = [data["test"]["answers"][idx]["text"][0] for idx in range(data["test"].shape[0])]

In [42]:
import evaluate
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=answer_pred, references=answer_true, lang="en")
# Embeddings bases
print(f"F1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

F1: 0.82337636500597, Precision: 0.8175586153353963, Recall: 0.8307474734527724


In [43]:
squad_v2_metric = evaluate.load("squad_v2")
references = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in zip(data["test"]["id"], data["test"]["answers"])]
predictions = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data["test"]["id"], answer_pred)]
results = squad_v2_metric.compute(predictions=predictions, references=references)
results

{'exact': 33.92857142857143,
 'f1': 44.40846181641183,
 'total': 56,
 'HasAns_exact': 33.92857142857143,
 'HasAns_f1': 44.40846181641183,
 'HasAns_total': 56,
 'best_exact': 33.92857142857143,
 'best_exact_thresh': 0.0,
 'best_f1': 44.40846181641183,
 'best_f1_thresh': 0.0}

In [44]:
bleu = evaluate.load("bleu")
references = [[answer] for answer in answer_true]
predictions = answer_pred
# N-Gram based
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.15766505662825686,
 'precisions': [0.2857142857142857,
  0.19066147859922178,
  0.12903225806451613,
  0.08791208791208792],
 'brevity_penalty': 1.0,
 'length_ratio': 1.4951456310679612,
 'translation_length': 308,
 'reference_length': 206}

In [45]:
model_no_ft = (AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased").to(device))

In [46]:
answer_pred_no_ft = [inference_answer(model_no_ft, data["test"]["question"][idx], data["test"]["context"][idx]) for idx in range(data["test"].shape[0])]

In [47]:
results = bertscore.compute(predictions=answer_pred_no_ft, references=answer_true, lang="en")
# Embeddings bases
print(f"F1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

F1: 0.3329433873295784, Precision: 0.32331089462552753, Recall: 0.34416832136256353


In [50]:
references = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [answer["text"][0]]}, "id": str(id)} for id, answer in zip(data["test"]["id"], data["test"]["answers"])]
predictions = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.} for id, answer in zip(data["test"]["id"], answer_pred_no_ft)]
results = squad_v2_metric.compute(predictions=predictions, references=references)
results

{'exact': 0.0,
 'f1': 4.7607317985469235,
 'total': 56,
 'HasAns_exact': 0.0,
 'HasAns_f1': 4.7607317985469235,
 'HasAns_total': 56,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 4.7607317985469235,
 'best_f1_thresh': 0.0}

In [52]:
references = [[answer] for answer in answer_true]
predictions = answer_pred_no_ft
# N-Gram based
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.016860049000712505,
 'precisions': [0.056768558951965066,
  0.022988505747126436,
  0.012135922330097087,
  0.00510204081632653],
 'brevity_penalty': 1.0,
 'length_ratio': 2.2233009708737863,
 'translation_length': 458,
 'reference_length': 206}